# Transportation Equilibrium Model: Deep Provenance Chains

This notebook demonstrates how Consist tracks provenance through an **iterative equilibrium model**—a common pattern in transportation, economics, and agent-based simulation where outputs feed back as inputs until the system converges.

## The Model

We simulate morning commute mode choice in a stylized 5-zone linear city:

```
Zone 1 ←→ Zone 2 ←→ Zone 3 ←→ Zone 4 ←→ Zone 5
(Low)     (Med)     (High/CBD)  (Med)     (Low)
```

**Feedback loop:**
1. **Logsums**: Compute accessibility from current travel times
2. **Trip Distribution**: Workers choose job locations based on accessibility
3. **Mode Choice**: Each commuter chooses car, transit, or walk
4. **Assignment**: Car trips create congestion
5. **Skim Update**: Congestion slows travel times → back to step 1

The model iterates until mode shares stabilize.

## Provenance Value

This structure lets us answer questions like:
- "Which iteration's skims were used to produce this trip table?"
- "If I change parking costs, which steps need to re-run?"
- "Can I resume from iteration 7 with a tighter convergence threshold?"


In [ ]:
from __future__ import annotations

import sys
from pathlib import Path


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists():
            return candidate
    raise RuntimeError("Could not locate repo root (missing pyproject.toml)")


REPO_ROOT = _find_repo_root(Path.cwd())
EXAMPLES_DIR = REPO_ROOT / "examples"
EXAMPLES_SRC = EXAMPLES_DIR / "src"

for path in (REPO_ROOT, EXAMPLES_SRC):
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

In [ ]:
from dataclasses import asdict, replace

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
from tqdm import tqdm

import consist
from consist import Tracker

from travel_demand_functions import (
    AssignmentParams,
    DestinationChoiceParams,
    ModeChoiceParams,
    TravelDemandScenarioConfig,
    ZoneParams,
    apply_congestion,
    apply_mode_choice,
    compute_mode_shares,
    compute_mode_utilities,
    compute_od_logsums,
    compute_od_volumes,
    create_skims_dataset,
    distribute_trips,
    generate_distances,
    generate_population,
    generate_zones,
    save_skims,
    summarize_iteration,
)

sns.set_theme(style="whitegrid")

## Setup

We initialize the tracker and import our model functions. Each run gets a unique session ID so we can compare multiple runs in the same database.

In [ ]:
EXAMPLES_DIR = REPO_ROOT / "examples"
RUN_DIR = EXAMPLES_DIR / "runs" / "travel_demand_demo"
SESSION_ID = os.getenv("CONSIST_SESSION_ID", "demo")
DB_PATH = RUN_DIR / f"travel_demand_demo_{SESSION_ID}.duckdb"
if DB_PATH.exists():
    DB_PATH.unlink()

tracker = Tracker(
    run_dir=RUN_DIR,
    db_path=DB_PATH,
    hashing_strategy="fast",
    project_root=str(RUN_DIR),
)

## Model Configuration

The model has four parameter groups:

| Parameter Group           | Controls                                                     |
|---------------------------|--------------------------------------------------------------|
| `ZoneParams`              | City layout: population, jobs, parking costs, transit access |
| `ModeChoiceParams`        | Utility coefficients, speeds, costs                          |
| `DestinationChoiceParams` | How accessibility affects job location choice                |
| `AssignmentParams`        | BPR congestion function parameters                           |

**ZoneParams** defines the city structure. The central zone (Zone 3) has the most jobs and highest parking costs, mimicking a downtown CBD. Outer zones are residential with cheaper or free parking.

**ModeChoiceParams** sets the coefficients for a standard multinomial logit mode choice model. `beta_cost` and `beta_time` control sensitivity to travel cost and time; the ASCs (alternative-specific constants) capture baseline preferences for transit and walking relative to driving.

**DestinationChoiceParams** governs how workers choose job locations. `beta_access` weights the logsum (a measure of how easy it is to reach a zone by any mode), while `beta_size` weights employment size.

**AssignmentParams** controls the BPR (Bureau of Public Roads) volume-delay function that converts traffic volumes into congested travel times.

We also set two equilibrium parameters:
- `SKIM_PERTURBATION`: Initial congestion multiplier (starts the system away from equilibrium so we can watch it converge)
- `DESTINATION_UPDATE_SHARE`: Damping factor for stability (blend new destination choices with previous iteration to prevent oscillation)

In [ ]:
zone_params = ZoneParams()
mode_params = ModeChoiceParams()
dest_params = DestinationChoiceParams()
assignment_params = AssignmentParams()

DEFAULT_SEED = 0

base_config = TravelDemandScenarioConfig(
    n_iterations=10,
    seed=DEFAULT_SEED,
    zone_params=zone_params,
    mode_params=mode_params,
    dest_params=dest_params,
    assignment_params=assignment_params,
)

SCENARIO_NAME = "travel_demand_demo"

SKIM_PERTURBATION = 2.0
DESTINATION_UPDATE_SHARE = 0.4

## Baseline Scenario

We run the equilibrium loop, tracking each step's inputs and outputs. The coupler passes artifacts between steps within the scenario—when we update `skims` after traffic simulation, the next iteration's logsum calculation automatically picks up the congested version.

### Step Structure per Iteration

```
┌─────────────┐
│    init     │ (iteration 0 only)
└──────┬──────┘
       │ skims, zones, population
       ▼
┌─────────────┐
│   logsums   │◄─── skims, zones
└──────┬──────┘
       │ logsums
       ▼
┌─────────────┐
│ trip_dist   │◄─── logsums, zones, population
└──────┬──────┘
       │ trips
       ▼
┌─────────────┐
│ utilities   │◄─── trips, skims, zones
└──────┬──────┘
       │ utilities
       ▼
┌─────────────┐
│ mode_choice │◄─── utilities, (prev trips_with_modes)
└──────┬──────┘
       │ trips_with_modes
       ▼
┌─────────────┐
│ assignment  │◄─── trips_with_modes
└──────┬──────┘
       │ volumes
       ▼
┌─────────────┐
│ traffic_sim │◄─── volumes, skims
└──────┬──────┘
       │ updated skims ──► next iteration
       ▼
```

In [ ]:
def run_scenario(scenario_config, scenario_run_id):
    mode_shares = {}
    summaries = {}
    pmt_totals = {}

    zone_params = scenario_config.zone_params
    mode_params = scenario_config.mode_params
    dest_params = scenario_config.dest_params
    assignment_params = scenario_config.assignment_params
    seed = scenario_config.seed

    with tracker.scenario(
        scenario_run_id,
        config={
            **asdict(scenario_config),
            "scenario_name": SCENARIO_NAME,
        },
        facet_from=["n_iterations"],
        tags=["examples", "simulation", "travel_demand"],
    ) as scenario:
        cache_validation = "lazy"  # Skip output checks on cache hits for speed; use 'eager' to validate files.
        cache_hydration = (
            "inputs-missing"  # Copy cached inputs into the new run_dir on cache misses.
        )

        def _init():
            zones = generate_zones(zone_params)
            population = generate_population(zones)
            distances = generate_distances(zone_params)

            skims = create_skims_dataset(zones, distances, mode_params)
            skims["time_car_mins"] *= SKIM_PERTURBATION

            output_dir = tracker.run_artifact_dir()
            output_dir.mkdir(parents=True, exist_ok=True)
            input_skims_path = output_dir / "skims_init.zarr"

            save_skims(skims, input_skims_path)

            consist.log_artifact(input_skims_path, key="skims", direction="output")
            consist.log_dataframe(
                population,
                key="persons",
                direction="output",
            )
            consist.log_dataframe(zones, key="zones", direction="output")

        scenario.run_step(
            name="init",
            fn=_init,
            config=asdict(mode_params)
            | asdict(zone_params)
            | {"skim_perturbation": SKIM_PERTURBATION},
            facet_from=["skim_perturbation"],
            outputs=["skims", "persons", "zones"],
        )

        for i in tqdm(range(scenario_config.n_iterations)):

            def _logsums():
                skims = consist.load(scenario.coupler.require("skims"))
                zones = consist.load(scenario.coupler.require("zones"))
                logsums = compute_od_logsums(skims, zones, mode_params)
                consist.log_dataframe(
                    logsums.to_dataframe().reset_index(),
                    key="logsums",
                )

            scenario.run_step(
                name="logsums",
                fn=_logsums,
                run_id=f"{scenario.run_id}_logsums_{i}",
                config=asdict(mode_params),
                input_keys=["skims", "zones"],
                facet_from=[
                    "beta_cost",
                    "beta_time",
                    "asc_walk",
                    "asc_transit",
                    "fuel_cost_per_mile",
                    "transit_fare",
                ],
                validate_cached_outputs=cache_validation,
                cache_hydration=cache_hydration,
                outputs=["logsums"],
                iteration=i,
            )

            def _trip_distribution():
                zones = consist.load(scenario.coupler.require("zones"))
                population = consist.load(scenario.coupler.require("persons"))
                prev_trips_art = scenario.coupler.get("trips")
                prev_trips = consist.load(prev_trips_art) if prev_trips_art else None
                logsums_df = consist.load(scenario.coupler.require("logsums"))
                if (
                    "origin" not in logsums_df.columns
                    or "destination" not in logsums_df.columns
                ):
                    logsums_df = logsums_df.reset_index()
                logsums = logsums_df.set_index(["origin", "destination"])[
                    "logsum"
                ].to_xarray()
                trips = distribute_trips(
                    population,
                    zones,
                    logsums,
                    dest_params,
                    seed=seed + i,
                    prev_trips=prev_trips,
                    update_share=DESTINATION_UPDATE_SHARE,
                )
                consist.log_dataframe(trips, key="trips")

            scenario.run_step(
                name="trip_distribution",
                fn=_trip_distribution,
                run_id=f"{scenario.run_id}_trip_distribution_{i}",
                config={
                    **asdict(dest_params),
                    "update_share": DESTINATION_UPDATE_SHARE,
                },
                input_keys=["skims", "zones", "persons", "logsums"],
                optional_input_keys=["trips"],
                facet_from=["beta_size", "beta_access", "update_share"],
                outputs=["trips"],
                iteration=i,
            )

            def _calculate_utilities():
                trips = consist.load(scenario.coupler.require("trips"))
                skims = consist.load(scenario.coupler.require("skims"))
                zones = consist.load(scenario.coupler.require("zones"))
                utilities = compute_mode_utilities(trips, skims, zones, mode_params)
                consist.log_dataframe(utilities, key="utilities")

            scenario.run_step(
                name="calculate_utilities",
                fn=_calculate_utilities,
                run_id=f"{scenario.run_id}_utilities_{i}",
                config=asdict(mode_params),
                input_keys=["trips", "skims", "zones"],
                facet_from=[
                    "beta_cost",
                    "beta_time",
                    "asc_walk",
                    "asc_transit",
                ],
                validate_cached_outputs=cache_validation,
                cache_hydration=cache_hydration,
                outputs=["utilities"],
                iteration=i,
            )

            def _mode_choice():
                utilities = consist.load(scenario.coupler.require("utilities"))
                trips_with_modes = apply_mode_choice(utilities, seed=seed + i)
                consist.log_dataframe(
                    trips_with_modes,
                    key="trips_with_modes",
                )

            scenario.run_step(
                name="mode_choice",
                fn=_mode_choice,
                run_id=f"{scenario.run_id}_mode_choice_{i}",
                config={"seed": seed},
                input_keys=["trips", "utilities"],
                outputs=["trips_with_modes"],
                iteration=i,
            )

            def _assignment():
                trips_with_modes = consist.load(
                    scenario.coupler.require("trips_with_modes")
                )
                volumes = compute_od_volumes(trips_with_modes)
                consist.log_dataframe(volumes, key="volumes")

            scenario.run_step(
                name="assignment",
                fn=_assignment,
                run_id=f"{scenario.run_id}_assignment_{i}",
                input_keys=["trips_with_modes"],
                outputs=["volumes"],
                iteration=i,
            )

            def _traffic_simulation():
                output_dir = tracker.run_artifact_dir()
                output_dir.mkdir(parents=True, exist_ok=True)
                volumes = consist.load(scenario.coupler.require("volumes"))
                skims = consist.load(scenario.coupler.require("skims"))
                skims = apply_congestion(skims, volumes, assignment_params)
                output_skims_path = output_dir / "skims_out.zarr"
                save_skims(skims, output_skims_path)
                consist.log_artifact(output_skims_path, key="skims", direction="output")

            scenario.run_step(
                name="traffic_simulation",
                fn=_traffic_simulation,
                run_id=f"{scenario.run_id}_traffic_simulation_{i}",
                config=asdict(assignment_params),
                input_keys=["volumes", "skims"],
                facet_from=["bpr_alpha", "bpr_beta", "base_capacity"],
                outputs=["skims"],
                iteration=i,
            )

            trips_with_modes = consist.load(
                scenario.coupler.require("trips_with_modes")
            )
            shares = compute_mode_shares(trips_with_modes)
            mode_shares[i] = pd.Series(shares)
            distance_by_mode = trips_with_modes.groupby("mode")["distance_miles"].sum()
            pmt_totals[i] = distance_by_mode.sort_index()
            summaries[i] = summarize_iteration(i, trips_with_modes, shares, 0, False)

        def _summaries():
            output_dir = tracker.run_artifact_dir()
            summary_dir = output_dir / "summary"
            summary_dir.mkdir(parents=True, exist_ok=True)

            mode_shares_df = (
                pd.DataFrame.from_dict(mode_shares, orient="index")
                .sort_index()
                .rename_axis("iteration")
                .reset_index()
            )
            summaries_df = (
                pd.DataFrame.from_dict(summaries, orient="index")
                .sort_values("iteration")
                .reset_index(drop=True)
            )
            pmt_totals_df = (
                pd.DataFrame.from_dict(pmt_totals, orient="index")
                .sort_index()
                .rename_axis("iteration")
                .reset_index()
            )

            mode_shares_path = summary_dir / "mode_shares.csv"
            summaries_path = summary_dir / "iteration_summaries.csv"
            pmt_totals_path = summary_dir / "pmt_totals.csv"

            shares_long = mode_shares_df.melt(
                id_vars="iteration", var_name="mode", value_name="share"
            )
            plt.figure(figsize=(8, 4))
            sns.lineplot(
                data=shares_long, x="iteration", y="share", hue="mode", marker="o"
            )
            plt.title("Mode Shares by Iteration")
            plt.tight_layout()
            mode_share_plot_path = summary_dir / "mode_shares.png"
            plt.savefig(mode_share_plot_path)
            plt.close()

            pmt_long = pmt_totals_df.melt(
                id_vars="iteration", var_name="mode", value_name="pmt"
            )
            fig, axes = plt.subplots(1, 2, figsize=(10, 4))
            sns.lineplot(
                data=summaries_df, x="iteration", y="vmt", marker="o", ax=axes[0]
            )
            axes[0].set_title("Vehicle Miles Traveled")
            sns.lineplot(
                data=pmt_long,
                x="iteration",
                y="pmt",
                hue="mode",
                marker="o",
                ax=axes[1],
            )
            axes[1].set_title("Person Miles Traveled by Mode")
            fig.tight_layout()
            totals_plot_path = summary_dir / "iteration_totals.png"
            fig.savefig(totals_plot_path)
            plt.close(fig)

            consist.log_dataframe(
                mode_shares_df,
                key="mode_shares",
                path=mode_shares_path,
            )
            consist.log_dataframe(
                summaries_df,
                key="iteration_summaries",
                path=summaries_path,
            )
            consist.log_dataframe(
                pmt_totals_df,
                key="pmt_totals",
                path=pmt_totals_path,
            )
            consist.log_artifact(
                mode_share_plot_path,
                key="mode_shares_plot",
                direction="output",
            )
            consist.log_artifact(
                totals_plot_path,
                key="iteration_totals_plot",
                direction="output",
            )

        scenario.run_step(
            name="summaries",
            fn=_summaries,
            input_keys=["trips_with_modes"],
            outputs=[
                "mode_shares",
                "iteration_summaries",
                "pmt_totals",
                "mode_shares_plot",
                "iteration_totals_plot",
            ],
            validate_cached_outputs=cache_validation,
            cache_hydration=cache_hydration,
        )

        mode_shares_df = consist.load(scenario.coupler.require("mode_shares"))
        summaries_df = consist.load(scenario.coupler.require("iteration_summaries"))
        pmt_totals_df = consist.load(scenario.coupler.require("pmt_totals"))
        mode_share_plot_path = scenario.coupler.path("mode_shares_plot")
        totals_plot_path = scenario.coupler.path("iteration_totals_plot")

    return {
        "scenario_run_id": scenario_run_id,
        "mode_shares_df": mode_shares_df,
        "summaries_df": summaries_df,
        "pmt_totals_df": pmt_totals_df,
        "mode_share_plot_path": mode_share_plot_path,
        "totals_plot_path": totals_plot_path,
    }

In [ ]:
base_run_id = f"{SCENARIO_NAME}_{SESSION_ID}"
base_results = run_scenario(base_config, base_run_id)

analysis = base_results

mode_shares_df = analysis["mode_shares_df"]
summaries_df = analysis["summaries_df"]
pmt_totals_df = analysis["pmt_totals_df"]
mode_share_plot_path = analysis["mode_share_plot_path"]
totals_plot_path = analysis["totals_plot_path"]
SCENARIO_RUN_ID = analysis["scenario_run_id"]

## Scenario Comparison: Higher Parking Costs

Now let's run a second scenario with higher CBD parking costs and compare the outcomes tothe baseline run. We'll focus on final-iteration mode shares, person-miles traveled, andcore summary metrics.


In [ ]:
high_parking_config = replace(
    base_config,
    zone_params=ZoneParams(parking_costs=(0.0, 8.0, 25.0, 8.0, 0.0)),
)
high_parking_run_id = f"{SCENARIO_NAME}_{SESSION_ID}_high_parking"
high_parking_results = run_scenario(high_parking_config, high_parking_run_id)

final_iteration = base_results["summaries_df"]["iteration"].max()

base_shares = base_results["mode_shares_df"].set_index("iteration").loc[final_iteration]
hp_shares = (
    high_parking_results["mode_shares_df"].set_index("iteration").loc[final_iteration]
)
mode_share_comparison = pd.DataFrame(
    {
        "base": base_shares,
        "high_parking": hp_shares,
    }
)
mode_share_comparison["delta"] = (
    mode_share_comparison["high_parking"] - mode_share_comparison["base"]
)

base_pmt = base_results["pmt_totals_df"].set_index("iteration").loc[final_iteration]
hp_pmt = (
    high_parking_results["pmt_totals_df"].set_index("iteration").loc[final_iteration]
)
pmt_comparison = pd.DataFrame(
    {
        "base": base_pmt,
        "high_parking": hp_pmt,
    }
)
pmt_comparison["delta"] = pmt_comparison["high_parking"] - pmt_comparison["base"]

summary_cols = ["vmt", "total_trips", "car_share", "transit_share", "walk_share"]
base_summary = (
    base_results["summaries_df"]
    .set_index("iteration")
    .loc[final_iteration, summary_cols]
)
hp_summary = (
    high_parking_results["summaries_df"]
    .set_index("iteration")
    .loc[final_iteration, summary_cols]
)
summary_comparison = pd.DataFrame(
    {
        "base": base_summary,
        "high_parking": hp_summary,
    }
)
summary_comparison["delta"] = (
    summary_comparison["high_parking"] - summary_comparison["base"]
)

display(mode_share_comparison)
display(pmt_comparison)
display(summary_comparison)

## Convergence Results

Let's examine how mode shares evolved across iterations. In a well-tuned model, shares should
stabilize as the feedback between congestion and mode choice reaches equilibrium.


In [ ]:
mode_shares_df

In [ ]:
from IPython.display import Image

Image(filename=str(mode_share_plot_path))

## Incremental Computation Demo

We ran the baseline scenario for 10 iterations above. Now we'll run an extended scenario for15 iterations. Since the extended run shares the same parameters for iterations 0-9, Consistshould recognize those steps have already been computed and skip them—only running iterations 10through 14.


In [10]:
import os

os.environ["CONSIST_CACHE_DEBUG"] = (
    "1"  # Log cache hits/misses during the incremental run.
)
extended_config = replace(base_config, n_iterations=15)
extended_run_id = f"{SCENARIO_NAME}_{SESSION_ID}_extended"
extended_results = run_scenario(extended_config, extended_run_id)

100%|██████████| 15/15 [01:29<00:00,  5.96s/it]


In [11]:
# Quick cache-hit check for the extended run.
cached_runs = [
    run
    for run in tracker.find_runs(parent_id=extended_results["scenario_run_id"])
    if run.meta.get("cache_hit")
]
[(run.model_name, run.iteration, run.id) for run in cached_runs]

[('traffic_simulation',
  9,
  'travel_demand_demo_demo_extended_traffic_simulation_9'),
 ('assignment', 9, 'travel_demand_demo_demo_extended_assignment_9'),
 ('mode_choice', 9, 'travel_demand_demo_demo_extended_mode_choice_9'),
 ('calculate_utilities', 9, 'travel_demand_demo_demo_extended_utilities_9'),
 ('trip_distribution',
  9,
  'travel_demand_demo_demo_extended_trip_distribution_9'),
 ('logsums', 9, 'travel_demand_demo_demo_extended_logsums_9'),
 ('traffic_simulation',
  8,
  'travel_demand_demo_demo_extended_traffic_simulation_8'),
 ('assignment', 8, 'travel_demand_demo_demo_extended_assignment_8'),
 ('mode_choice', 8, 'travel_demand_demo_demo_extended_mode_choice_8'),
 ('calculate_utilities', 8, 'travel_demand_demo_demo_extended_utilities_8'),
 ('trip_distribution',
  8,
  'travel_demand_demo_demo_extended_trip_distribution_8'),
 ('logsums', 8, 'travel_demand_demo_demo_extended_logsums_8'),
 ('traffic_simulation',
  7,
  'travel_demand_demo_demo_extended_traffic_simulation_7'),

As expected, all steps from iterations 0-9 were retrieved from cache. The extended run only
computed the 5 new iterations, demonstrating how Consist enables incremental refinement of
iterative models.


## Querying Provenance

Now let's use Consist's query capabilities to explore the artifact lineage. We can find all the model runs in a given iteration


In [12]:
# Runs for a specific iteration (e.g., iteration 5).
iteration_runs = [
    run
    for run in tracker.find_runs(parent_id=SCENARIO_RUN_ID, status="completed")
    if run.iteration == 5
]
iteration_run_df = pd.DataFrame(
    [
        {
            "model": run.model_name,
            "run_id": run.id,
            "iteration": run.iteration,
        }
        for run in iteration_runs
    ]
)
iteration_run_df

,model,run_id,iteration
0,traffic_simulation,travel_demand_demo_demo_traffic_simulation_5,5
1,assignment,travel_demand_demo_demo_assignment_5,5
2,mode_choice,travel_demand_demo_demo_mode_choice_5,5
3,calculate_utilities,travel_demand_demo_demo_utilities_5,5
4,trip_distribution,travel_demand_demo_demo_trip_distribution_5,5
5,logsums,travel_demand_demo_demo_logsums_5,5


We can find all the artifacts created during that iteration

In [13]:
iter5_artifacts = {
    run.id: [artifact.key for artifact in tracker.find_artifacts(creator=run)]
    for run in iteration_runs
}
iter5_artifacts

{'travel_demand_demo_demo_traffic_simulation_5': ['skims'],
 'travel_demand_demo_demo_assignment_5': ['volumes'],
 'travel_demand_demo_demo_mode_choice_5': ['trips_with_modes'],
 'travel_demand_demo_demo_utilities_5': ['utilities'],
 'travel_demand_demo_demo_trip_distribution_5': ['trips'],
 'travel_demand_demo_demo_logsums_5': ['logsums']}

### Artifact Lineage

The lineage tree shows how the final `skims` artifact traces back through the computational graph. Reading the tree:

- Each **artifact** (like `skims`, `volumes`) shows its key and unique ID
- Below each artifact is the **run** that produced it, with the step name, run ID, and iteration number
- Indented below each run are its **input artifacts**, which recursively show their own producers

This forms a DAG (directed acyclic graph) where you can trace any output back to the original inputs. With `max_depth=4`, we see four levels of this chain—in a 10-iteration model, the full lineage would be much deeper.

In [14]:
from rich import print as rprint
from rich.tree import Tree


def _add_lineage(branch, node):
    artifact = node["artifact"]
    art_label = f"{artifact.key} ({artifact.id})"
    art_branch = branch.add(art_label)
    run_node = node.get("producing_run")
    if not run_node:
        return
    run = run_node["run"]
    run_label = f"{run.model_name} run={run.id} iter={run.iteration}"
    run_branch = art_branch.add(run_label)
    for child in run_node.get("inputs", []):
        _add_lineage(run_branch, child)


final_skims = tracker.get_artifact("skims")
lineage = (
    tracker.get_artifact_lineage(final_skims.id, max_depth=4) if final_skims else None
)

if lineage:
    tree = Tree("lineage")
    _add_lineage(tree, lineage)
    rprint(tree)

lineage
└── skims (3b3e7c46-4f09-4ed2-b3e7-6011b4d8c02c)
    └── traffic_simulation run=travel_demand_demo_demo_extended_traffic_simulation_14 iter=14
        ├── skims (daea68d0-aa98-4f78-9328-88293edb49c5)
        │   └── traffic_simulation run=travel_demand_demo_demo_extended_traffic_simulation_13 iter=13
        │       ├── skims (4206abfe-ea82-4ef2-bf3f-bd94f8c4181f)
        │       │   └── traffic_simulation run=travel_demand_demo_demo_extended_traffic_simulation_12 iter=12
        │       │       ├── skims (c0710b41-3511-40a1-a26e-94f0dcd4d64d)
        │       │       │   └── traffic_simulation run=travel_demand_demo_demo_extended_traffic_simulation_11 
        │       │       │       iter=11
        │       │       │       ├── skims (60f24ff4-9ecd-4f7c-8307-893d20b49140)
        │       │       │       └── volumes (c214bcff-82c8-4d2e-91f0-91d8dcf9ed42)
        │       │       └── volumes (42c9f231-b8d7-476d-b47b-0cc7d6faafe2)
        │       │           └── assignment run=travel_demand_demo_demo_extended_assignment_12 iter=12
        │       │               └── trips_with_modes (9889e58f-7e67-4ab7-95ef-a81d10e7f9ef)
        │       └── volumes (769a4dcb-f072-40e7-95a3-1e8cd8e20ca0)
        │           └── assignment run=travel_demand_demo_demo_extended_assignment_13 iter=13
        │               └── trips_with_modes (585fbabc-c5a5-45e3-9e46-8913e0dd464c)
        │                   └── mode_choice run=travel_demand_demo_demo_extended_mode_choice_13 iter=13
        │                       ├── trips (6654a93b-0e7b-4121-9f87-e03f0272aa64)
        │                       └── utilities (bc527dfb-5d62-4cae-a742-75cad7395482)
        └── volumes (aac3bb18-6b4e-4309-a63e-90fb9668ed2e)
            └── assignment run=travel_demand_demo_demo_extended_assignment_14 iter=14
                └── trips_with_modes (cd99c62f-c923-4bec-bdc8-10504d3b2b39)
                    └── mode_choice run=travel_demand_demo_demo_extended_mode_choice_14 iter=14
                        ├── trips (cbab71fd-1069-49d1-a088-19f8b9a9fbaa)
                        │   └── trip_distribution run=travel_demand_demo_demo_extended_trip_distribution_14 iter=14
                        │       ├── persons (e7a3f67a-de39-4ea5-8b9d-0a9d569be0ad)
                        │       ├── zones (48da5433-19fa-49a0-aa4e-01a5017df18e)
                        │       ├── trips (6654a93b-0e7b-4121-9f87-e03f0272aa64)
                        │       ├── skims (daea68d0-aa98-4f78-9328-88293edb49c5)
                        │       └── logsums (47668c88-c9b4-4b2c-bb07-22d4aa7fd4c5)
                        └── utilities (c4b6d9d7-2895-4467-883e-07129aa3fe95)
                            └── calculate_utilities run=travel_demand_demo_demo_extended_utilities_14 iter=14
                                ├── zones (48da5433-19fa-49a0-aa4e-01a5017df18e)
                                ├── skims (daea68d0-aa98-4f78-9328-88293edb49c5)
                                └── trips (cbab71fd-1069-49d1-a088-19f8b9a9fbaa)

In [15]:
# Which steps would re-run if parking costs changed?
# Parking costs live in the zones artifact, so any run that consumes 'zones'
# would be invalidated.
zone_consumers = []
for run in tracker.find_runs(parent_id=SCENARIO_RUN_ID, status="completed"):
    artifacts = tracker.get_artifacts_for_run(run.id)
    if "zones" in artifacts.inputs:
        zone_consumers.append(run)
zone_consumer_df = pd.DataFrame(
    [
        {"model": run.model_name, "run_id": run.id, "iteration": run.iteration}
        for run in zone_consumers
    ]
)
zone_consumer_df

,model,run_id,iteration
0,calculate_utilities,travel_demand_demo_demo_utilities_9,9
1,trip_distribution,travel_demand_demo_demo_trip_distribution_9,9
2,logsums,travel_demand_demo_demo_logsums_9,9
3,calculate_utilities,travel_demand_demo_demo_utilities_8,8
4,trip_distribution,travel_demand_demo_demo_trip_distribution_8,8
5,logsums,travel_demand_demo_demo_logsums_8,8
6,calculate_utilities,travel_demand_demo_demo_utilities_7,7
7,trip_distribution,travel_demand_demo_demo_trip_distribution_7,7
8,logsums,travel_demand_demo_demo_logsums_7,7
9,calculate_utilities,travel_demand_demo_demo_utilities_6,6


## Summary

This notebook demonstrated three key provenance patterns:

1. **Deep chains**: Each iteration's outputs become the next iteration's inputs, creating
   traceable lineage dozens of steps deep.
2. **Incremental computation**: Running 15 iterations after already running 10 reused all
   prior work—only the new iterations executed.
3. **Impact analysis**: We can query which steps consume a given artifact to understand
   what would need to re-run if that artifact changed.

The transportation model is a toy example, but the patterns apply to any iterative workflow:
MCMC sampling, neural network training checkpoints, agent-based simulations, economic
equilibrium models.
